# Import

In [18]:
import os
import numpy as np
import cv2
import pydicom as dicom
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

# Path

In [19]:
project_path = r"D:\Liver160"
model_path = r"D:\Liver160\files\model.h5"


# Utilities

In [20]:
def create_dir(path):
    path = os.path.join(project_path, path)
    if not os.path.exists(path):
        os.makedirs(path)


In [21]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


In [22]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)


In [23]:
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


# Predict

In [86]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("dicom_results")

""" Loading model """
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
  model = tf.keras.models.load_model(model_path)

""" Load the dataset """
test_x = glob(os.path.join(project_path, "dicom_test/*.dcm"))
print(f"Test: {len(test_x)}")

""" Loop over the data """
for x in tqdm(test_x):
  """ Extract the names """
  dir_name = x.split(os.sep)[-3]
  name = dir_name + "_" + x.split(os.sep)[-1].split(".")[0]

  """ Read the image """
  image = dicom.dcmread(x).pixel_array
  image = cv2.resize(image, (160,160))
  image = np.expand_dims(image, axis=-1)
  image = image + 1024 
  image = image / np.max(image) 
  image = image * 255


  x = image / 255 #(160,160,1)
  x = np.concatenate([x, x, x], axis=-1) #(160,160,3)
  # x = cv2.resize(x, (160,160))
  x = np.expand_dims(x, axis=0) #(1,160,160,3)


  
  

  
  


  """ Prediction """
  mask = model.predict(x)[0]
  mask = mask > 0.5
  mask = mask.astype(np.int32)
  mask = mask * 255
  # print(mask.shape) (160,160,1)
  cat_images = np.concatenate([image, mask], axis=1)
  image_path = os.path.join(project_path ,"dicom_results", f"{name}")
  cv2.imwrite(f"{image_path}.png", cat_images)


Test: 280


  0%|          | 0/280 [00:00<?, ?it/s]

1/1 [==============================] - 0s 385ms/step


  0%|          | 1/280 [00:00<02:02,  2.27it/s]

1/1 [==============================] - 0s 62ms/step


...

 99%|█████████▉| 277/280 [00:28<00:00, 10.35it/s]

1/1 [==============================] - 0s 65ms/step


100%|█████████▉| 279/280 [00:28<00:00, 10.22it/s]

1/1 [==============================] - 0s 62ms/step


100%|██████████| 280/280 [00:28<00:00,  9.87it/s]
